In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import warnings

In [2]:
%matplotlib qt
warnings.filterwarnings('ignore')

In [21]:
N=100; # number of grid boxes
L=1; # length of the domain
x=np.linspace(-L/2,L/2,N);
dx=x[1]-x[0];
T0= 1/np.sqrt(2*np.pi)/dx*np.exp(-(x/dx)**2/2); #width of the Gaussian is dx, and its
                                            # intergral is 1. So in the limit of dx->0 this is a delta-Function.
T=T0;
T_theory = T0;
T_dirich = T0 # new!
T_neu = T0 # new!
D=0.01; # diffusivity coefficient
# Courant stability requires 0.5 prefactor minimum
dt=0.25*dx**2/D;
total_time=2; #simulation time
Nt=np.fix(total_time/dt); # number of timesteps
Txx=np.zeros(len(T));
Txx_dirich = np.zeros(len(T))
Txx_neu = np.zeros(len(T))

fig,axs = plt.subplots()

line1 = axs.plot(x,T, label = 'Simulation')[0]
line2 = axs.plot(x,T_theory, label = 'Theory')[0]
dirichlet_line = axs.plot(x, T_dirich, label = 'Dirichlet B.C.')[0] # new!
neu_line = axs.plot(x, T_neu, label='Neumann B.C.')[0] # new!
plt.legend()


def update(frame):
    global T,Txx # make sure we can update T within the function
    global T_dirich, Txx_dirich
    global T_neu, Txx_neu

    time=frame*dt;
    T_theory=1/np.sqrt(4*np.pi*D*time)*np.exp(-x**2/(4*D*time));
    #plotting
    # calculating Txx using a simple central difference scheme, O(dx^2)
    Txx[1:-1]=(T[2:]-2*T[1:-1] + T[:-2])/dx**2;

    #periodic b.c.
    Txx[0]=(T[1]-2*T[0] + T[-1])/dx**2;
    Txx[-1]=(T[0]-2*T[-1] + T[-1])/dx**2;

    # Dirichlet boundary conditions, new!
    Txx_dirich[1:-1] = (T_dirich[2:] - 2*T_dirich[1:-1] + T_dirich[:-2])/dx**2
    T_dirich[0] = 0
    T_dirich[-1] = 0

    # Neumann boundary conditions
    Txx_neu[1:-1]=(T_neu[2:]-2*T_neu[1:-1] + T_neu[:-2])/dx**2;
    T_neu[0] = T_neu[1]
    T_neu[-1] = T_neu[-2]

    # updating temperature (time-stepping)
    T=T+D*Txx*dt;
    T_dirich = T_dirich + D*Txx_dirich*dt # new!
    T_neu = T_neu + D*Txx_neu*dt
    line1.set_ydata(T)
    line2.set_ydata(T_theory)
    dirichlet_line.set_ydata(T_dirich) # new!
    neu_line.set_ydata(T_neu)
    axs.set_ylim(0,np.max(T)*1.1)
    return line1,line2, dirichlet_line

    
# Create animation
ani = animation.FuncAnimation(fig, update, frames=int(Nt), interval=1,repeat=False)

# Show the animation
plt.tight_layout()
plt.show()
